### Pipeline

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"
BASE_LAYER_DIR = r"D:\Columbia\Fall2025\5400\project\layer"

BRONZE_PATH = os.path.join(BASE_LAYER_DIR, "bronze")
SILVER_PATH = os.path.join(BASE_LAYER_DIR, "silver")
GOLD_PATH   = os.path.join(BASE_LAYER_DIR, "gold")

def create_spark(app_name="5400-news-elt"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        # 内存给足一点，避免大 CSV 写 parquet 时崩
        .config("spark.driver.memory", "8g")
        .config("spark.executor.memory", "8g")
        # 提高并行度（根据你 CPU 核心数可调）
        .config("spark.sql.shuffle.partitions", "200")
        .config("spark.default.parallelism", "200")
        # parquet 压缩
        .config("spark.sql.parquet.compression.codec", "snappy")
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    print("✅ Spark version:", spark.version)
    return spark


bronze

In [8]:
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"
BASE_LAYER_DIR = r"D:\Columbia\Fall2025\5400\project\layer"
BRONZE_PATH = os.path.join(BASE_LAYER_DIR, "bronze")

def write_bronze(spark):
    os.makedirs(BRONZE_PATH, exist_ok=True)

    df = (
        spark.read
        .format("csv")
        .options(
            header="true",
            inferSchema="true",
            multiLine="true",        # 保险：文章可能跨行
            escape="\"",
            quote="\"",
            mode="PERMISSIVE"
        )
        .load(CSV_PATH)
    )

    print("📊 CSV rows (rough):", df.count())

    # 轻微清洗：去掉两边空格
    string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, T.StringType)]
    for c in string_cols:
        df = df.withColumn(c, F.trim(F.col(c)))

    # Date 强制成日期格式（如果已经是 date 类型，这步会自动适配）
    if "Date" in df.columns:
        df = df.withColumn(
            "Date",
            F.to_date("Date", "yyyy-MM-dd")
        )

    # 为了更均衡的 parquet 分区，按 Stock_symbol 重分区（避免单文件过大）
    if "Stock_symbol" in df.columns:
        df = df.repartition(200, "Stock_symbol")
    else:
        df = df.repartition(200)

    (
        df.write
        .mode("overwrite")
        .parquet(BRONZE_PATH)
    )

    print("✅ Bronze 层写入完成：", BRONZE_PATH)
    return df


In [5]:
SILVER_PATH = os.path.join(BASE_LAYER_DIR, "silver")

def build_silver(spark):
    os.makedirs(SILVER_PATH, exist_ok=True)

    bronze_df = spark.read.parquet(BRONZE_PATH)

    df = bronze_df

    # 业务上很关键的列：Date + Stock_symbol 缺失的直接扔掉
    if "Date" in df.columns:
        df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns:
        df = df.filter(F.col("Stock_symbol").isNotNull())

    # 去掉完全重复行
    df = df.dropDuplicates()

    # 标准化 Publisher / Author 的大小写（示例）
    if "Publisher" in df.columns:
        df = df.withColumn("Publisher_norm", F.upper(F.col("Publisher")))
    if "Author" in df.columns:
        df = df.withColumn("Author_norm", F.upper(F.col("Author")))

    # 给每条新闻一个唯一 ID（方便以后 join）
    df = df.withColumn(
        "news_id",
        F.monotonically_increasing_id()
    )

    # 再按 (Stock_symbol, Date) 重新分区，写入 silver
    if {"Stock_symbol", "Date"}.issubset(df.columns):
        df = df.repartition(200, "Stock_symbol", "Date")
    else:
        df = df.repartition(200)

    (
        df.write
        .mode("overwrite")
        .parquet(SILVER_PATH)
    )

    print("✅ Silver 层写入完成：", SILVER_PATH)
    return df


In [9]:
GOLD_PATH = os.path.join(BASE_LAYER_DIR, "gold")

def build_gold(spark):
    os.makedirs(GOLD_PATH, exist_ok=True)

    silver_df = spark.read.parquet(SILVER_PATH)

    # 保证关键字段存在
    needed_cols = ["Date", "Stock_symbol"]
    for c in needed_cols:
        if c not in silver_df.columns:
            raise ValueError(f"Silver 缺少必要字段：{c}")

    df = silver_df

    # 标题长度
    if "Article_title" in df.columns:
        df = df.withColumn("title_len", F.length(F.col("Article_title")))
    else:
        df = df.withColumn("title_len", F.lit(None).cast("int"))

    group_cols = ["Date", "Stock_symbol"]

    agg_exprs = [
        F.count("*").alias("article_count"),
        F.countDistinct("Publisher").alias("publisher_count"),
        F.avg("title_len").alias("avg_title_len"),
    ]

    # 选一个 summary 字段作为代表（有哪个用哪个）
    summary_cols = [
        "Lsa_summary",
        "Luhn_summary",
        "Textrank_summary",
        "Lexrank_summary"
    ]
    for sc in summary_cols:
        if sc in df.columns:
            agg_exprs.append(F.first(sc, ignorenulls=True).alias(f"sample_{sc}"))

    gold_df = (
        df.groupBy(*group_cols)
        .agg(*agg_exprs)
    )

    # 再按 Stock_symbol 分区存储，方便按股票读取
    gold_df = gold_df.repartition(200, "Stock_symbol")

    (
        gold_df.write
        .mode("overwrite")
        .parquet(GOLD_PATH)
    )

    print("✅ Gold 层写入完成：", GOLD_PATH)
    return gold_df


In [10]:
if __name__ == "__main__":
    spark = create_spark()

    # A1: Bronze
    write_bronze(spark)

    # A2: Silver
    #build_silver(spark)

    # A3: Gold
    #build_gold(spark)

    #spark.stop()


✅ Spark version: 3.3.1
📊 CSV rows (rough): 13057514
✅ Bronze 层写入完成： D:\Columbia\Fall2025\5400\project\layer\bronze


In [12]:
if __name__ == "__main__":
    spark = create_spark()

    # A2: Silver
    build_silver(spark)

✅ Spark version: 3.3.1
✅ Silver 层写入完成： D:\Columbia\Fall2025\5400\project\layer\silver


In [13]:
if __name__ == "__main__":
    spark = create_spark()

    # A3: Gold
    build_gold(spark)

✅ Spark version: 3.3.1
✅ Gold 层写入完成： D:\Columbia\Fall2025\5400\project\layer\gold


In [3]:
if __name__ == "__main__":
    spark = create_spark()

import os

from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType, StringType

silver_path = r"D:\Columbia\Fall2025\5400\project\layer\silver"

silver_df = spark.read.parquet(silver_path)
print("Silver count:", silver_df.count())


✅ Spark version: 3.3.1
Silver count: 3242351


In [4]:

# 选一层给 NLP 用，这里用 silver 层
nlp_layer_path = os.path.join(BASE_LAYER_DIR, "silver_for_nlp")

(
    silver_df
    .select(
        "Date",
        "Article_title",
        "Stock_symbol",
        "Publisher",
        "Author",
        "Url",
        "Article"
    )
    .write.mode("overwrite").parquet(nlp_layer_path)
)

print("已写出 NLP 输入层到：", nlp_layer_path)


已写出 NLP 输入层到： D:\Columbia\Fall2025\5400\project\layer\silver_for_nlp


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
GPU name: NVIDIA GeForce RTX 5070 Ti Laptop GPU


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda")
emb = model.encode("hello world")
print(type(emb), len(emb))


D:\Code\ANACONDA3\envs\pyspark1\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

D:\Code\ANACONDA3\envs\pyspark1\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [7]:
import transformers
print("Transformers version:", transformers.__version__)

Transformers version: 4.41.2


In [4]:
import os
os.environ["HF_HOME"] = r"D:\Columbia\huggingface_cache"

from transformers import pipeline
from sentence_transformers import SentenceTransformer
import torch

# 1. Sentiment pipeline
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="finiteautomata/bertweet-base-sentiment-analysis",
    top_k=None
)

# 2. Embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 3. Summarization pipeline
summary_pipe = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6"
)


config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # 禁用 GPU，让 transformers 只用 CPU
os.environ["HF_HOME"] = r"D:\Columbia\huggingface_cache"  # 保证在 import 之前

from transformers import pipeline
import torch

device = -1
print("device =", device)

# ① 情感分析（英文）
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=device
)

# ② 摘要模型 —— 换成小的 DistilBART，而不是 facebook/bart-large-cnn
summ_pipe = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=device
)

txt = "The market fell sharply today, but some analysts remain optimistic about the long-term outlook."
print("sentiment:", sentiment_pipe(txt))
print("summary:", summ_pipe(txt, max_length=40, min_length=10, truncation=True)[0]["summary_text"])


device = -1


Your max_length is set to 40, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


sentiment: [{'label': 'POSITIVE', 'score': 0.9784604907035828}]
summary:  The market fell sharply today, but some analysts remain optimistic about the long-term outlook .


In [8]:
bronze_path = r"D:\Columbia\Fall2025\5400\project\layer\bronze"

news_df = spark.read.parquet(bronze_path)
news_df.printSchema()
news_df.select("Article").show(5, truncate=80)


root
 |-- Date: date (nullable = true)
 |-- Article_title: string (nullable = true)
 |-- Stock_symbol: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Author: string (nullable = true)
 |-- Article: string (nullable = true)
 |-- Lsa_summary: string (nullable = true)
 |-- Luhn_summary: string (nullable = true)
 |-- Textrank_summary: string (nullable = true)
 |-- Lexrank_summary: string (nullable = true)

+-------+
|Article|
+-------+
|   null|
|   null|
|   null|
|   null|
|   null|
+-------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

@pandas_udf("string")
def udf_summary(texts: pd.Series) -> pd.Series:
    def one(t):
        if not isinstance(t, str) or not t.strip():
            return None
        try:
            # 文本太长要截断，否则 summarization 会报错或非常慢
            t_cut = t[:2000]
            res = summ_pipe(t_cut, max_length=80, min_length=20, truncation=True)[0]
            return res["summary_text"]
        except Exception:
            return None
    return texts.apply(one)

@pandas_udf("string")
def udf_sentiment(texts: pd.Series) -> pd.Series:
    def one(t):
        if not isinstance(t, str) or not t.strip():
            return None
        try:
            t_cut = t[:512]  # sentiment 模型输入也截一下
            res = sentiment_pipe(t_cut)[0]  # [{'label': 'POSITIVE', 'score': 0.99}]
            return f"{res['label']}|{res['score']:.4f}"
        except Exception:
            return None
    return texts.apply(one)


In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""   # 禁用 GPU，避免 CUDA 报错
os.environ["HF_HOME"] = r"D:\Columbia\huggingface_cache"

from transformers import pipeline
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# ---------- 1. 全局 pipeline（每个 Python worker 只初始化一次） ----------

sentiment_pipe = None
summ_pipe = None

def get_sentiment_pipe():
    global sentiment_pipe
    if sentiment_pipe is None:
        sentiment_pipe = pipeline(
            "sentiment-analysis",
            model="distilbert-base-uncased-finetuned-sst-2-english",
            device=-1,              # 只用 CPU
        )
    return sentiment_pipe

def get_summ_pipe():
    global summ_pipe
    if summ_pipe is None:
        summ_pipe = pipeline(
            "summarization",
            model="sshleifer/distilbart-cnn-12-6",
            device=-1,              # 只用 CPU
        )
    return summ_pipe

# ---------- 2. UDF 函数 ----------

def summarize_text(text: str) -> str:
    if text is None:
        return None
    text = text.strip()
    if not text:
        return None
    pipe = get_summ_pipe()
    out = pipe(
        text,
        max_length=80,   # 可以根据你文章长度再调
        min_length=15,
        truncation=True
    )[0]["summary_text"]
    return out

def sentiment_text(text: str) -> str:
    if text is None:
        return None
    text = text.strip()
    if not text:
        return None
    pipe = get_sentiment_pipe()
    res = pipe(text)[0]
    # 返回 "POSITIVE(0.98)" 这种
    return f"{res['label']}({res['score']:.3f})"

# ---------- 3. 注册成 Spark UDF ----------

udf_summary = udf(summarize_text, StringType())
udf_sentiment = udf(sentiment_text, StringType())


In [16]:
# 例如 news_df 里文章列叫 "Article"
sample_df = news_df.limit(500)   # 先 500 行试试

enriched_df = (
    sample_df
        .withColumn("article_summary", udf_summary("Article"))
        .withColumn("article_sentiment", udf_sentiment("Article"))
)

enriched_df.select(
    "Article", "article_summary", "article_sentiment"
).show(5, truncate=80)


Py4JJavaError: An error occurred while calling o88.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 404) (10.206.89.237 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:581)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:91)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:68)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(Interruptibl

In [1]:
import site
site.getsitepackages()


['E:\\envall\\all.venv',
 'E:\\envall\\all.venv\\Lib\\site-packages',
 'C:\\Users\\GUTS\\AppData\\Local\\Programs\\Python\\Python311',
 'C:\\Users\\GUTS\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages']

In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability())


True
NVIDIA GeForce RTX 5070 Ti Laptop GPU
(12, 0)


In [15]:
sample_df = news_df.limit(1)  # 只做 100 行 demo

enriched_df = (
    sample_df
    .withColumn("article_summary", udf_summary("Article"))
    .withColumn("article_sentiment", udf_sentiment("Article"))
)

enriched_df.select("Article", "article_summary", "article_sentiment")\
           .show(5, truncate=80)


Py4JJavaError: An error occurred while calling o155.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 995) (10.206.89.237 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:581)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:107)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(

In [ ]:
def sentiment_udf(text):
    if text is None or len(text.strip()) == 0:
        return ("neutral", 0.0)
    try:
        result = sentiment_pipe(text[:512])[0]    # 限制512字符，避免模型爆
        return (result["label"], float(result["score"]))
    except:
        return ("neutral", 0.0)

sentiment_spark_udf = F.udf(sentiment_udf,
                            returnType=F.struct(
                                F.col("label").cast("string"),
                                F.col("score").cast("double")
                            ))
